## Rotation of the coefficient matrix of the molecular orbital for different basis sets 

The molecular orbitals are constructed from atomic orbitals using the following linear summation:

$$
\Psi_{i} = \sum \limits_{j} C_{ij} \phi_{j}
$$

The coefficient matrix $C$ is the matrix that contains the coefficients of the linear combination of atomic orbitals that form the molecular orbitals. The coefficient matrix is different for each basis set. $\phi_{\mu j}$ is the atomic orbital $\mu$ for atom $j$ and contains a product of radial and angular (spherical harmonic) parts. 

In this notebook, we understand the rotation of the coefficient matrix computed in different basis sets. To illustrate this rotation, we will use a simple example of a water molecule, rotated at different Euler angles $\alpha$, $\beta$ and $\gamma$.

In [ ]:
from collections import defaultdict

import json

import numpy as np

import plotly.express as px

from ase.data import atomic_numbers
from ase.visualize import view

from pymatgen.core.structure import Molecule
from pymatgen.io.ase import AseAtomsAdaptor

import basis_set_exchange as bse

import torch

from e3nn import o3

from minimal_basis.predata.matrices import ReducedBasisMatrices 
from minimal_basis.transforms.rotations import RotationMatrix

from instance_mongodb import instance_mongodb_sei

In [ ]:
db = instance_mongodb_sei(project="mlts")
collection = db.rotated_water_calculations
basis_sets = collection.distinct('orig.rem.basis')
data = defaultdict(lambda: defaultdict(list))

for basis_set in basis_sets:
    basis_info = bse.get_basis(basis_set, fmt='json', elements=[1, 8])
    basis_info = json.loads(basis_info)

    electron_shells = {k: basis_info['elements'][k]['electron_shells'] for k in basis_info['elements'].keys()}
    angular_momenta = {k: [shell['angular_momentum'] for shell in electron_shells[k]] for k in electron_shells.keys()}
    angular_momenta = {k: [item for sublist in angular_momenta[k] for item in sublist] for k in angular_momenta.keys()}

    find_tags = {'orig.rem.basis': basis_set, 'tags.inverted_coordinates': True}
    for doc in collection.find(find_tags).sort('tags.idx', 1):
        _alpha_coeff_matrix = doc['calcs_reversed'][0]['alpha_coeff_matrix']
        _alpha_eigenvalues = doc['calcs_reversed'][0]['alpha_eigenvalues']
        _alpha_fock_matrix = doc['calcs_reversed'][0]['alpha_fock_matrix']

        _alpha_coeff_matrix = np.array(_alpha_coeff_matrix)
        _alpha_eigenvalues = np.array(_alpha_eigenvalues)
        _alpha_fock_matrix = np.array(_alpha_fock_matrix)

        molecule = Molecule.from_dict(doc['orig']['molecule'])
        symbols = [site.specie.symbol for site in molecule.sites]
        atom_numbers = [atomic_numbers[symbol] for symbol in symbols]

        _irreps = ""
        indices_to_keep = []
        _idx = 0
        _basis_functions_orbital = []
        for atom_number in atom_numbers:
            _angular_momenta = angular_momenta[str(atom_number)]
            _angular_momenta = np.array(_angular_momenta)
            _basis_functions = 2*_angular_momenta + 1
            for _basis_function in _basis_functions:
                if _basis_function == 1:
                    _basis_functions_orbital.extend(['s'])
                    _irreps += "+1x0e"
                    indices_to_keep.append(_idx)
                    _idx += 1
                elif _basis_function == 3:
                    _basis_functions_orbital.extend(['p', 'p', 'p'])
                    _irreps += "+1x1o"
                    indices_to_keep.extend([_idx+1, _idx+2, _idx])
                    _idx += 3
                elif _basis_function == 5 and basis_set != '6-31g*':
                    _basis_functions_orbital.extend(['d', 'd', 'd', 'd', 'd'])
                    _irreps += "+1x2e"
                    indices_to_keep.extend([_idx, _idx+1, _idx+2, _idx+3, _idx+4]) 
                    _idx += 5
                elif _basis_function == 5 and basis_set == '6-31g*':
                    _idx += 6

        _irreps = _irreps[1:]

        base_quantities_qchem = ReducedBasisMatrices(
            fock_matrix=_alpha_fock_matrix,
            eigenvalues=_alpha_eigenvalues,
            coeff_matrix=_alpha_coeff_matrix,
            indices_to_keep=indices_to_keep,
        )

        alpha_ortho_coeff_matrix = base_quantities_qchem.get_ortho_coeff_matrix()
        alpha_coeff_matrix = base_quantities_qchem.get_coeff_matrix()
        alpha_fock_matrix = base_quantities_qchem.get_fock_matrix()
        alpha_eigenvalues = base_quantities_qchem.eigenval_ortho_fock


        data[basis_set]['alpha_coeff_matrix'].append(alpha_coeff_matrix)
        data[basis_set]['alpha_eigenvalues'].append(alpha_eigenvalues)
        data[basis_set]['alpha_fock_matrix'].append(alpha_fock_matrix)
        data[basis_set]['alpha_ortho_coeff_matrix'].append(alpha_ortho_coeff_matrix)
        data[basis_set]['basis_functions_orbital'].append(_basis_functions_orbital)
        data[basis_set]['irreps'].append(_irreps)
        data[basis_set]['euler_angles'].append(doc['tags']['euler_angles'])
        data[basis_set]['idx'].append(doc['tags']['idx'])
        data[basis_set]['molecule'].append(molecule)

for basis_set in basis_sets:
    for key in data[basis_set].keys():
        if key == 'molecule':
            continue
        data[basis_set][key] = np.array(data[basis_set][key])

In [ ]:
basis_set = "6-31g*"
quantity = "alpha_ortho_coeff_matrix"

## Structures

In [ ]:
all_molecules = data[basis_set]['molecule']
all_atoms = [AseAtomsAdaptor.get_atoms(molecule) for molecule in all_molecules]
# Vary the below index from 0-9 to visualise the rotations
view(all_atoms[7], viewer="x3d")

## $\alpha$ coefficient matrix

We will now generate the Coefficient matrices from DFT and compare it with the rotations coming from the Euler angles.

In [ ]:
fig = px.imshow(data[basis_set][quantity], animation_frame=0, range_color=[-1, 1])
labels = data[basis_set]['basis_functions_orbital'][0]
fig.update_yaxes(ticktext=labels, tickvals=list(range(len(labels))))
fig.update_xaxes(ticktext=np.round(data[basis_set]['alpha_eigenvalues'][0], 3), tickvals=list(range(len(data[basis_set]['alpha_eigenvalues'][0]))))
fig.update_yaxes(title_text="Atomic orbitals")
fig.update_xaxes(title_text="Eigenvalues of molecular orbitals (Ha)")
fig.update_layout(title="Coefficient matrix from DFT calculation")
fig.show()

In [ ]:
new_coeff_matrix = []
calculated_coeff_matrix = data[basis_set][quantity]
original_coeff_matrix = calculated_coeff_matrix[0] 

for _idx, angles in enumerate(data[basis_set]['euler_angles']):

    torch_angles = torch.tensor(angles, dtype=torch.float64)

    irreps = data[basis_set]['irreps'][_idx]
    irreps = o3.Irreps(irreps)

    rotation_matrix = RotationMatrix(angle_type="euler", angles=angles)() 
    rotation_matrix = torch.tensor(rotation_matrix, dtype=torch.float64)
     
    D_matrix = irreps.D_from_matrix(rotation_matrix)
    D_matrix = D_matrix.detach().numpy()

    _new_coeff_matrix = np.zeros_like(original_coeff_matrix)
    for i in range(original_coeff_matrix.shape[1]):
        _new_coeff_matrix[:, i] = original_coeff_matrix[:, i] @ D_matrix.T

    new_coeff_matrix.append(_new_coeff_matrix)

new_coeff_matrix = np.array(new_coeff_matrix)

fig = px.imshow(new_coeff_matrix, animation_frame=0, range_color=[-1, 1])
fig.update_yaxes(ticktext=labels, tickvals=list(range(len(labels))))
fig.update_xaxes(ticktext=np.round(data[basis_set]['alpha_eigenvalues'][0], 3), tickvals=list(range(len(data[basis_set]['alpha_eigenvalues'][0]))))
fig.update_yaxes(title_text="Atomic orbitals")
fig.update_xaxes(title_text="Eigenvalues of molecular orbitals (Ha)")
fig.update_layout(title="Rotated coefficient matrix from D-matrix based on rotated Euler angles")
fig.show()

In [ ]:
difference_matrices = np.abs(new_coeff_matrix) - np.abs(calculated_coeff_matrix)
fig = px.imshow(difference_matrices, animation_frame=0, range_color=[-1, 1])
fig.update_yaxes(ticktext=labels, tickvals=list(range(len(labels))))
fig.update_xaxes(ticktext=np.round(data[basis_set]['alpha_eigenvalues'][0], 3), tickvals=list(range(len(data[basis_set]['alpha_eigenvalues'][0]))))
fig.update_yaxes(title_text="Atomic orbitals")
fig.update_xaxes(title_text="Eigenvalues of molecular orbitals (Ha)")
fig.update_layout(title="Difference between rotated and calculated coefficient matrix")
fig.show()